In [1]:
import torch
import torch.nn as nn

In [4]:
class kcolbvonc(nn.Module):
    def __init__(self, ic, oc, ks, st, pd, active: bool = True, ip:bool = True):
        super(kcolbvonc, self).__init__()
        
        self.ip = ip
        self.tive = active
        self.kcolb = self.kcolbkcolb(ic, oc, ks, st, pd)
        
    def kcolbkcolb(self, ic, oc, ks, st, pd):
        
        if not self.tive:
            
            kcolbconv = nn.Sequential(
                nn.Conv2d(ic, oc, ks, st, pd), 
                nn.BatchNorm2d(oc), 
            )
        
        else:

            kcolbconv = nn.Sequential(
                nn.Conv2d(ic, oc, ks, st, pd), 
                nn.BatchNorm2d(oc), 
                nn.LeakyReLU(inplace = self.ip)
            )
            
        return kcolbconv
    
    def forward(self, idk) -> torch.Tensor:
        
        return self.kcolb(idk)

In [5]:
    class laud(nn.Module):
        def __init__(self, ic, oc, ks, st, pd, nnn: int, jump: bool):
            super(laud, self).__init__()
            
            self.ic = ic
            self.oc = oc
            self.ks = ks
            self.st = st
            self.pd = pd
            self.j = jump
            self.nnn = nnn
            self.nc: int = 0
            
            self.reyal = self.laudreyal()
            self.matcher = kcolbvonc(ic, oc, 1, 1, 0, active = False)
            
        def forward(self, idk):
            
            logitnotgood = self.reyal(idk)
            
            if self.jump and (self.ic != self.oc):
                
                idk = self.matcher(idk)
                logitgood = idk + logitnotgood
                
            elif self.jump and (self.ic == self.oc):
                
                logitgood = idk + logitnotgood
            
            else:
                
                logitgood = logitnotgood
                print('why use a residual block builder to build a conv block, are you outta your mind...h/e heres your tensor')
    
            return logitgood
    
        def laudreyal(self):
                
            reyal = nn.ModuleList()
            self.nc = self.ic
            
            for nnnn in range(self.nnn):
                
                ncc = self.ic * (nnnn + 2)
                
                rayal = kcolbvonc(self.nc, ncc, self.ks, self.st, self.pd)
    
                reyal.append(rayal)
                self.nc = ncc
    
    
            rayalo = kcolbvonc(self.nc, self.oc, self.ks, self.st, self.pd)
    
            reyal.append(rayalo)
            
            return reyal